In [1]:
# =========================
# 0) Install (RELIABLE)
# =========================
!pip -q uninstall -y autogen pyautogen ag2 >/dev/null 2>&1
!pip -q install -U "ag2[openai]" python-dotenv

!pip show ag2
!python -c "import autogen; print('autogen version:', getattr(autogen,'__version__', 'unknown'))"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.7/936.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.5 MB/s eta 0:00:00
Name: ag2
Version: 0.10.3
Summary: A programming framework for agentic AI
Home-page: https://ag2.ai/
Author: 
Author-email: Chi Wang & Qingyun Wu <support@ag2.ai>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: anyio, diskcache, docker, httpx, packaging, pydantic, python-dotenv, termcolor, tiktoken
Required-by: 
autogen version: 0.10.3


In [ ]:
# ============================================================
# 1) Env (DO NOT hardcode keys)
#   Colab: Runtime > Secrets > add OPENAI_API_KEY
# ============================================================
import os, json, time, re, hashlib, sqlite3, sys
from dotenv import load_dotenv
load_dotenv()

# Read from Colab Secrets / .env
os.environ["OPENAI_API_KEY"] =   # set via Secrets
os.environ["OPENAI_MODEL_NAME"] = os.environ.get("OPENAI_MODEL_NAME", "gpt-4o-mini")

print("OPENAI_MODEL_NAME:", os.environ["OPENAI_MODEL_NAME"])
print("OPENAI_API_KEY present?", bool(os.getenv("OPENAI_API_KEY")))

# ============================================================
# 2) Mount Drive + File Path(s)
# ============================================================
from google.colab import drive

FILE_PATH = "/content/drive/MyDrive/Network Maintenace - With & without maintenace/Original-data/2017-2019 (no main).csv"
csv_paths = [FILE_PATH]

if not os.path.exists("/content/drive/MyDrive"):
    drive.mount("/content/drive", force_remount=True)

for p in csv_paths:
    if not os.path.exists(p):
        raise FileNotFoundError(f"CSV path not found:\n{p}")

print("Mounted?", os.path.exists("/content/drive/MyDrive"))
print("CSV paths exist?", all(os.path.exists(p) for p in csv_paths))


OPENAI_MODEL_NAME: gpt-4o-mini
OPENAI_API_KEY present? True
Mounted at /content/drive
Mounted? True
CSV paths exist? True


In [3]:
# ============================================================
# 3) Workspace + Hybrid Persistent Memory (Notebook version)
#   Then EXPORT to mm_runtime.py (AutoGen-safe)
# ============================================================
from typing import Optional, List, Dict, Any, Tuple
import numpy as np
import pandas as pd
import networkx as nx

WORK_DIR = "/content/pavement_agentic_workspace"
os.makedirs(WORK_DIR, exist_ok=True)

MEM_PATH      = os.path.join(WORK_DIR, "memory.jsonl")
DB_PATH       = os.path.join(WORK_DIR, "memory_hybrid.sqlite")
VEC_PATH      = os.path.join(WORK_DIR, "memory_vectors.npz")
KG_PATH       = os.path.join(WORK_DIR, "knowledge_graph.graphml")
REGISTRY_PATH = os.path.join(WORK_DIR, "file_path_registry.json")

# -------------------------
# JSONL audit
# -------------------------
def _mem_load_raw() -> List[Dict[str, Any]]:
    if not os.path.exists(MEM_PATH):
        return []
    rows = []
    with open(MEM_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except Exception:
                pass
    return rows

def mem_add_jsonl(kind: str, text: str, meta: Optional[dict] = None) -> str:
    meta = meta or {}
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    raw = f"{ts}|{kind}|{text}|{json.dumps(meta, sort_keys=True)}".encode("utf-8")
    memory_id = hashlib.sha256(raw).hexdigest()[:16]
    rec = {"id": memory_id, "ts": ts, "kind": str(kind), "text": str(text).strip(), "meta": meta}
    os.makedirs(WORK_DIR, exist_ok=True)
    with open(MEM_PATH, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    return memory_id

# -------------------------
# SQLite hybrid store
# -------------------------
def _db_connect() -> sqlite3.Connection:
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA journal_mode=WAL;")

    conn.execute(
        "CREATE TABLE IF NOT EXISTS memory_struct ("
        "id TEXT PRIMARY KEY, ts TEXT, kind TEXT, text TEXT, meta_json TEXT"
        ");"
    )
    conn.execute(
        "CREATE VIRTUAL TABLE IF NOT EXISTS memory_fts USING fts5("
        "id UNINDEXED, kind, text, content='', tokenize='porter'"
        ");"
    )
    conn.execute(
        "CREATE TABLE IF NOT EXISTS doc_summaries ("
        "doc_id TEXT PRIMARY KEY, title TEXT, file_path TEXT, summary TEXT, meta_json TEXT"
        ");"
    )
    conn.execute(
        "CREATE TABLE IF NOT EXISTS nodes ("
        "node_id TEXT PRIMARY KEY, node_type TEXT, title TEXT, text TEXT, doc_id TEXT, meta_json TEXT"
        ");"
    )
    conn.execute(
        "CREATE TABLE IF NOT EXISTS edges ("
        "src TEXT, rel TEXT, dst TEXT, meta_json TEXT, PRIMARY KEY (src, rel, dst)"
        ");"
    )

    conn.commit()
    return conn

def keyword_index_add(memory_id: str, ts: str, kind: str, text: str, meta: dict):
    conn = _db_connect()
    conn.execute(
        "INSERT OR REPLACE INTO memory_struct(id, ts, kind, text, meta_json) VALUES (?, ?, ?, ?, ?)",
        (memory_id, ts, kind, text, json.dumps(meta, ensure_ascii=False))
    )
    conn.execute("INSERT INTO memory_fts(id, kind, text) VALUES (?, ?, ?)", (memory_id, kind, text))
    conn.commit()
    conn.close()

def keyword_search(query: str, k: int = 5, kind: Optional[str] = None) -> List[Dict[str, Any]]:
    conn = _db_connect()
    where_kind = "AND kind = ?" if kind else ""
    params = [query]
    if kind:
        params.append(kind)
    params.append(k)

    sql = (
        "SELECT id, kind, text, bm25(memory_fts) AS score "
        "FROM memory_fts "
        "WHERE memory_fts MATCH ? "
        f"{where_kind} "
        "ORDER BY score "
        "LIMIT ?;"
    )
    rows = conn.execute(sql, params).fetchall()
    conn.close()

    out = []
    for rid, rkind, rtext, score in rows:
        out.append({"id": rid, "kind": rkind, "text": rtext, "score": float(-score)})
    return out

# -------------------------
# Semantic vectors (NPZ)
# -------------------------
def _load_vec_store() -> Tuple[List[str], np.ndarray]:
    if not os.path.exists(VEC_PATH):
        return [], np.zeros((0, 0), dtype=np.float32)
    data = np.load(VEC_PATH, allow_pickle=True)
    return data["ids"].tolist(), data["vecs"].astype(np.float32)

def _save_vec_store(ids: List[str], vecs: np.ndarray):
    np.savez(VEC_PATH, ids=np.array(ids, dtype=object), vecs=vecs.astype(np.float32))

def _normalize_rows(X: np.ndarray) -> np.ndarray:
    if X.size == 0:
        return X
    norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    return X / norms

def embed_text(text: str) -> np.ndarray:
    try:
        from sentence_transformers import SentenceTransformer
        global _ST_MODEL
        if "_ST_MODEL" not in globals():
            _ST_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
        v = _ST_MODEL.encode([text], normalize_embeddings=True)[0]
        return np.array(v, dtype=np.float32)
    except Exception:
        dim = 512
        v = np.zeros(dim, dtype=np.float32)
        for token in text.lower().split():
            h = int(hashlib.sha256(token.encode()).hexdigest(), 16)
            v[h % dim] += 1.0
        v = v / (np.linalg.norm(v) + 1e-12)
        return v

def semantic_add(memory_id: str, text: str):
    ids, vecs = _load_vec_store()
    v = embed_text(text).reshape(1, -1)

    if vecs.size == 0:
        ids = [memory_id]
        vecs = v
    else:
        if v.shape[1] != vecs.shape[1]:
            raise ValueError(f"Embedding dim mismatch: got {v.shape[1]} vs store {vecs.shape[1]}")
        ids.append(memory_id)
        vecs = np.vstack([vecs, v])

    _save_vec_store(ids, vecs)

def semantic_search(query: str, k: int = 5, kind: Optional[str] = None) -> List[Dict[str, Any]]:
    ids, vecs = _load_vec_store()
    if vecs.size == 0:
        return []

    q = embed_text(query).reshape(1, -1).astype(np.float32)
    vecs_n = _normalize_rows(vecs)
    q_n = _normalize_rows(q)
    sims = (vecs_n @ q_n.T).ravel()
    top = np.argsort(-sims)[: min(k * 5, len(ids))]

    # kind filter using memory_struct (optional)
    id2kind = None
    if kind and len(top) > 0:
        conn = _db_connect()
        ph = ",".join(["?"] * len(top))
        rows = conn.execute(
            f"SELECT id, kind FROM memory_struct WHERE id IN ({ph})",
            [ids[i] for i in top]
        ).fetchall()
        conn.close()
        id2kind = {rid: rk for rid, rk in rows}

    out = []
    for i in top:
        rid = ids[i]
        if kind and id2kind and (id2kind.get(rid) != kind):
            continue
        out.append({"id": rid, "score": float(sims[i])})
        if len(out) >= k:
            break
    return out

# -------------------------
# KG (NetworkX)
# -------------------------
def _kg_load() -> nx.MultiDiGraph:
    if os.path.exists(KG_PATH):
        try:
            g = nx.read_graphml(KG_PATH)
            mg = nx.MultiDiGraph()
            mg.add_nodes_from(g.nodes(data=True))
            mg.add_edges_from(g.edges(data=True))
            return mg
        except Exception:
            pass
    return nx.MultiDiGraph()

_KG = _kg_load()

def _kg_save():
    g = nx.DiGraph()
    g.add_nodes_from(_KG.nodes(data=True))
    for u, v, data in _KG.edges(data=True):
        g.add_edge(u, v, **data)
    nx.write_graphml(g, KG_PATH)

def kg_add_fact(subj: str, pred: str, obj: str, confidence: float = 1.0, meta: Optional[dict] = None, index_to_memory: bool = True):
    meta = meta or {}
    subj = str(subj); pred = str(pred); obj = str(obj)

    _KG.add_node(subj)
    _KG.add_node(obj)
    _KG.add_edge(subj, obj, relation=pred, confidence=float(confidence), **meta)
    _kg_save()

    if index_to_memory:
        mem_add_jsonl("kg_fact", f"KG FACT: ({subj}) -[{pred}]-> ({obj}) conf={confidence}",
                      {"subj": subj, "pred": pred, "obj": obj, "confidence": float(confidence), **meta})

# -------------------------
# Registry + artifacts
# -------------------------
def _stable_dataset_id(file_path: str) -> str:
    st = os.stat(file_path)
    raw = f"{file_path}|{st.st_size}|{int(st.st_mtime)}".encode("utf-8")
    return hashlib.sha256(raw).hexdigest()[:12]

def registry_build_from_csv_paths(csv_paths: List[str]) -> Dict[str, Any]:
    mapping = {p: _stable_dataset_id(p) for p in csv_paths}
    reg = {
        "created_ts": time.strftime("%Y-%m-%d %H:%M:%S"),
        "path_to_dataset_id": mapping,
        "latest_path": list(mapping.keys())[-1],
        "latest_dataset_id": list(mapping.values())[-1],
    }
    with open(REGISTRY_PATH, "w", encoding="utf-8") as f:
        json.dump(reg, f, ensure_ascii=False, indent=2)
    return reg

def registry_load() -> Optional[Dict[str, Any]]:
    if not os.path.exists(REGISTRY_PATH):
        return None
    try:
        with open(REGISTRY_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return None

def store_dataset_artifacts(file_path: str, dataset_id: str):
    df = pd.read_csv(file_path)
    artifacts = {
        "dataset_id": dataset_id,
        "file_path": file_path,
        "shape": list(df.shape),
        "columns": list(df.columns),
        "dtypes": {c: str(df[c].dtype) for c in df.columns},
        "head": df.head(5).to_dict(orient="records"),
    }
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    text = f"DATASET COLUMNS {dataset_id}: " + ", ".join(list(df.columns)[:80])
    mid = mem_add_jsonl("dataset_artifacts", text, {"dataset_id": dataset_id, "file_path": file_path, "artifacts": artifacts})
    keyword_index_add(mid, ts, "dataset_artifacts", text, {"dataset_id": dataset_id, "file_path": file_path})
    try:
        semantic_add(mid, text)
    except Exception:
        pass

    kg_add_fact(f"dataset:{dataset_id}", "file_path", file_path, confidence=1.0, meta={}, index_to_memory=False)
    for c in df.columns:
        kg_add_fact(f"dataset:{dataset_id}", "has_column", c, confidence=1.0, meta={}, index_to_memory=False)

# -------------------------
# mm wrapper (Notebook)
# -------------------------
class MM:
    def health_check(self) -> bool:
        os.makedirs(WORK_DIR, exist_ok=True)
        if not os.path.exists(MEM_PATH):
            open(MEM_PATH, "a", encoding="utf-8").close()
        conn = _db_connect()
        conn.close()
        return True

    def rag_add(self, kind: str, text: str, meta: Optional[dict] = None):
        meta = meta or {}
        ts = time.strftime("%Y-%m-%d %H:%M:%S")
        mid = mem_add_jsonl(kind, text, meta)
        keyword_index_add(mid, ts, kind, text, meta)
        try:
            semantic_add(mid, text)
        except Exception:
            pass
        return True

    def rag_search(self, query: str, k: int = 5, kind: Optional[str] = None, alpha: float = 0.65):
        kw = keyword_search(query, k=k*2, kind=kind)
        sem = semantic_search(query, k=k*2, kind=kind)

        scores = {}
        for r in kw:
            scores.setdefault(r["id"], {"kw": 0.0, "sem": 0.0})
            scores[r["id"]]["kw"] = max(scores[r["id"]]["kw"], r["score"])
        for r in sem:
            scores.setdefault(r["id"], {"kw": 0.0, "sem": 0.0})
            scores[r["id"]]["sem"] = max(scores[r["id"]]["sem"], r["score"])

        if not scores:
            return []

        kw_vals = np.array([v["kw"] for v in scores.values()], dtype=np.float32)
        sem_vals = np.array([v["sem"] for v in scores.values()], dtype=np.float32)

        kw_norm = (kw_vals - kw_vals.min()) / (kw_vals.max() - kw_vals.min() + 1e-12) if kw_vals.size else kw_vals
        sem_norm = (sem_vals - sem_vals.min()) / (sem_vals.max() - sem_vals.min() + 1e-12) if sem_vals.size else sem_vals

        ids = list(scores.keys())
        for i, rid in enumerate(ids):
            scores[rid]["hybrid"] = float(alpha * kw_norm[i] + (1 - alpha) * sem_norm[i])

        ranked = sorted(ids, key=lambda rid: scores[rid]["hybrid"], reverse=True)[:k]
        all_rows = _mem_load_raw()
        id_to_row = {r.get("id"): r for r in all_rows}

        return [(scores[rid]["hybrid"], id_to_row.get(rid, {"id": rid, "kind": kind, "text": ""})) for rid in ranked]

    def rag_get_latest_registry(self) -> Dict[str, Any]:
        reg = registry_load()
        if reg is None:
            raise RuntimeError(f"Registry not found at {REGISTRY_PATH}. Run registry_build_from_csv_paths(csv_paths).")
        return reg

    def kg_add_fact(self, subj: str, pred: str, obj: str, confidence: float = 1.0, meta: Optional[dict] = None, index_to_memory: bool = True):
        kg_add_fact(subj, pred, obj, confidence=confidence, meta=meta, index_to_memory=index_to_memory)
        return True

mm = MM()
print("mm.health_check():", mm.health_check())
print("WORK_DIR:", WORK_DIR)

# ============================================================
# 3B) Export mm_runtime.py (AutoGen-safe)
#   IMPORTANT: no triple quotes inside this module string.
# ============================================================
import importlib

MM_MODULE_PATH = os.path.join(WORK_DIR, "mm_runtime.py")

mm_runtime_code = r'''
# Auto-generated hybrid memory module for AutoGen executor
# Usage:
#   from mm_runtime import mm

import os, json, time, hashlib, sqlite3
from typing import Optional, List, Dict, Any, Tuple
import numpy as np
import networkx as nx

WORK_DIR = r"__WORK_DIR__"
MEM_PATH      = os.path.join(WORK_DIR, "memory.jsonl")
DB_PATH       = os.path.join(WORK_DIR, "memory_hybrid.sqlite")
VEC_PATH      = os.path.join(WORK_DIR, "memory_vectors.npz")
KG_PATH       = os.path.join(WORK_DIR, "knowledge_graph.graphml")
REGISTRY_PATH = os.path.join(WORK_DIR, "file_path_registry.json")

def _mem_load_raw() -> List[Dict[str, Any]]:
    if not os.path.exists(MEM_PATH):
        return []
    rows = []
    with open(MEM_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except Exception:
                pass
    return rows

def mem_add_jsonl(kind: str, text: str, meta: Optional[dict] = None) -> str:
    meta = meta or {}
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    raw = f"{ts}|{kind}|{text}|{json.dumps(meta, sort_keys=True)}".encode("utf-8")
    memory_id = hashlib.sha256(raw).hexdigest()[:16]
    rec = {"id": memory_id, "ts": ts, "kind": str(kind), "text": str(text).strip(), "meta": meta}
    os.makedirs(WORK_DIR, exist_ok=True)
    if not os.path.exists(MEM_PATH):
        open(MEM_PATH, "a", encoding="utf-8").close()
    with open(MEM_PATH, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    return memory_id

def _db_connect() -> sqlite3.Connection:
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA journal_mode=WAL;")

    conn.execute("CREATE TABLE IF NOT EXISTS memory_struct (id TEXT PRIMARY KEY, ts TEXT, kind TEXT, text TEXT, meta_json TEXT);")
    conn.execute("CREATE VIRTUAL TABLE IF NOT EXISTS memory_fts USING fts5(id UNINDEXED, kind, text, content='', tokenize='porter');")
    conn.execute("CREATE TABLE IF NOT EXISTS doc_summaries (doc_id TEXT PRIMARY KEY, title TEXT, file_path TEXT, summary TEXT, meta_json TEXT);")
    conn.execute("CREATE TABLE IF NOT EXISTS nodes (node_id TEXT PRIMARY KEY, node_type TEXT, title TEXT, text TEXT, doc_id TEXT, meta_json TEXT);")
    conn.execute("CREATE TABLE IF NOT EXISTS edges (src TEXT, rel TEXT, dst TEXT, meta_json TEXT, PRIMARY KEY (src, rel, dst));")

    conn.commit()
    return conn

def keyword_index_add(memory_id: str, ts: str, kind: str, text: str, meta: dict):
    conn = _db_connect()
    conn.execute(
        "INSERT OR REPLACE INTO memory_struct(id, ts, kind, text, meta_json) VALUES (?, ?, ?, ?, ?)",
        (memory_id, ts, kind, text, json.dumps(meta, ensure_ascii=False))
    )
    conn.execute("INSERT INTO memory_fts(id, kind, text) VALUES (?, ?, ?)", (memory_id, kind, text))
    conn.commit()
    conn.close()

def keyword_search(query: str, k: int = 5, kind: Optional[str] = None) -> List[Dict[str, Any]]:
    conn = _db_connect()
    where_kind = " AND kind = ? " if kind else " "
    params = [query]
    if kind:
        params.append(kind)
    params.append(k)

    sql = (
        "SELECT id, kind, text, bm25(memory_fts) AS score "
        "FROM memory_fts "
        "WHERE memory_fts MATCH ? "
        + where_kind +
        "ORDER BY score "
        "LIMIT ?;"
    )
    rows = conn.execute(sql, params).fetchall()
    conn.close()
    return [{"id": rid, "kind": rkind, "text": rtext, "score": float(-score)} for rid, rkind, rtext, score in rows]

def _load_vec_store() -> Tuple[List[str], np.ndarray]:
    if not os.path.exists(VEC_PATH):
        return [], np.zeros((0, 0), dtype=np.float32)
    data = np.load(VEC_PATH, allow_pickle=True)
    return data["ids"].tolist(), data["vecs"].astype(np.float32)

def _save_vec_store(ids: List[str], vecs: np.ndarray):
    np.savez(VEC_PATH, ids=np.array(ids, dtype=object), vecs=vecs.astype(np.float32))

def _normalize_rows(X: np.ndarray) -> np.ndarray:
    if X.size == 0:
        return X
    norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    return X / norms

def embed_text(text: str) -> np.ndarray:
    try:
        from sentence_transformers import SentenceTransformer
        global _ST_MODEL
        if "_ST_MODEL" not in globals():
            _ST_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
        v = _ST_MODEL.encode([text], normalize_embeddings=True)[0]
        return np.array(v, dtype=np.float32)
    except Exception:
        dim = 512
        v = np.zeros(dim, dtype=np.float32)
        for token in text.lower().split():
            h = int(hashlib.sha256(token.encode()).hexdigest(), 16)
            v[h % dim] += 1.0
        v = v / (np.linalg.norm(v) + 1e-12)
        return v

def semantic_add(memory_id: str, text: str):
    ids, vecs = _load_vec_store()
    v = embed_text(text).reshape(1, -1)
    if vecs.size == 0:
        ids = [memory_id]
        vecs = v
    else:
        if v.shape[1] != vecs.shape[1]:
            raise ValueError(f"Embedding dim mismatch: got {v.shape[1]} vs store {vecs.shape[1]}")
        ids.append(memory_id)
        vecs = np.vstack([vecs, v])
    _save_vec_store(ids, vecs)

def semantic_search(query: str, k: int = 5, kind: Optional[str] = None) -> List[Dict[str, Any]]:
    ids, vecs = _load_vec_store()
    if vecs.size == 0:
        return []
    q = embed_text(query).reshape(1, -1).astype(np.float32)
    vecs_n = _normalize_rows(vecs)
    q_n = _normalize_rows(q)
    sims = (vecs_n @ q_n.T).ravel()
    top = np.argsort(-sims)[: min(k * 5, len(ids))]

    id2kind = None
    if kind and len(top) > 0:
        conn = _db_connect()
        ph = ",".join(["?"] * len(top))
        rows = conn.execute(f"SELECT id, kind FROM memory_struct WHERE id IN ({ph})", [ids[i] for i in top]).fetchall()
        conn.close()
        id2kind = {rid: rk for rid, rk in rows}

    out = []
    for i in top:
        rid = ids[i]
        if kind and id2kind and (id2kind.get(rid) != kind):
            continue
        out.append({"id": rid, "score": float(sims[i])})
        if len(out) >= k:
            break
    return out

def registry_load() -> Optional[Dict[str, Any]]:
    if not os.path.exists(REGISTRY_PATH):
        return None
    try:
        with open(REGISTRY_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return None

def health_check() -> bool:
    os.makedirs(WORK_DIR, exist_ok=True)
    if not os.path.exists(MEM_PATH):
        open(MEM_PATH, "a", encoding="utf-8").close()
    conn = _db_connect()
    conn.close()
    return True

class MM:
    def health_check(self) -> bool:
        return health_check()

    def rag_add(self, kind: str, text: str, meta: Optional[dict] = None):
        meta = meta or {}
        ts = time.strftime("%Y-%m-%d %H:%M:%S")
        mid = mem_add_jsonl(kind, text, meta)
        keyword_index_add(mid, ts, kind, text, meta)
        try:
            semantic_add(mid, text)
        except Exception:
            pass
        return True

    def rag_search(self, query: str, k: int = 5, kind: Optional[str] = None, alpha: float = 0.65):
        kw = keyword_search(query, k=k*2, kind=kind)
        sem = semantic_search(query, k=k*2, kind=kind)

        scores = {}
        for r in kw:
            scores.setdefault(r["id"], {"kw": 0.0, "sem": 0.0})
            scores[r["id"]]["kw"] = max(scores[r["id"]]["kw"], r["score"])
        for r in sem:
            scores.setdefault(r["id"], {"kw": 0.0, "sem": 0.0})
            scores[r["id"]]["sem"] = max(scores[r["id"]]["sem"], r["score"])

        if not scores:
            return []

        kw_vals = np.array([v["kw"] for v in scores.values()], dtype=np.float32)
        sem_vals = np.array([v["sem"] for v in scores.values()], dtype=np.float32)

        kw_norm = (kw_vals - kw_vals.min()) / (kw_vals.max() - kw_vals.min() + 1e-12) if kw_vals.size else kw_vals
        sem_norm = (sem_vals - sem_vals.min()) / (sem_vals.max() - sem_vals.min() + 1e-12) if sem_vals.size else sem_vals

        ids = list(scores.keys())
        for i, rid in enumerate(ids):
            scores[rid]["hybrid"] = float(alpha * kw_norm[i] + (1 - alpha) * sem_norm[i])

        ranked = sorted(ids, key=lambda rid: scores[rid]["hybrid"], reverse=True)[:k]
        all_rows = _mem_load_raw()
        id_to_row = {r.get("id"): r for r in all_rows}
        return [(scores[rid]["hybrid"], id_to_row.get(rid, {"id": rid, "kind": kind, "text": ""})) for rid in ranked]

    def rag_get_latest_registry(self) -> Dict[str, Any]:
        reg = registry_load()
        if reg is None:
            raise RuntimeError(f"Registry not found at {REGISTRY_PATH}. Build it in the notebook first.")
        return reg

mm = MM()
'''

mm_runtime_code = mm_runtime_code.replace("__WORK_DIR__", WORK_DIR)

with open(MM_MODULE_PATH, "w", encoding="utf-8") as f:
    f.write(mm_runtime_code)

print("✅ mm_runtime.py written to:", MM_MODULE_PATH)

if WORK_DIR not in sys.path:
    sys.path.append(WORK_DIR)

import mm_runtime
importlib.reload(mm_runtime)
print("✅ mm_runtime.mm.health_check():", mm_runtime.mm.health_check())

# ============================================================
# 4) Build registry + store artifacts
# ============================================================
reg = registry_build_from_csv_paths(csv_paths)
for p, dsid in reg["path_to_dataset_id"].items():
    store_dataset_artifacts(p, dsid)

mm.rag_add("registry", f"Registry built with {len(reg['path_to_dataset_id'])} datasets.", reg)
mm.rag_add("run_start", f"Run started. CSV files: {csv_paths}", {"csvs": csv_paths})

print("Latest dataset:", reg["latest_dataset_id"])
print("Latest path:", reg["latest_path"])

# ============================================================
# 5) AutoGen / ag2 setup (YOUR agent lineup)
# ============================================================
from autogen import ConversableAgent, UserProxyAgent, GroupChat, GroupChatManager

llm_config = {
    "config_list": [{"model": os.environ["OPENAI_MODEL_NAME"], "api_key": os.environ["OPENAI_API_KEY"]}],
    "temperature": 0.2,
}

user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    default_auto_reply="NEXT",
    max_consecutive_auto_reply=3,
    code_execution_config={"work_dir": WORK_DIR, "use_docker": False, "timeout": 900, "last_n_messages": 12},
)

# ---------------------------
# Persistent workspace paths
# ---------------------------
WORK_DIR = "/content/pavement_agentic_workspace"
os.makedirs(WORK_DIR, exist_ok=True)

MEM_PATH = os.path.join(WORK_DIR, "memory.jsonl")
REGISTRY_PATH = os.path.join(WORK_DIR, "file_path_registry.json")
KG_PATH = os.path.join(WORK_DIR, "knowledge_graph.graphml")

# ---------------------------
# MEMORY INSTRUCTIONS (for all agents)
# ---------------------------
MEMORY_INSTRUCTIONS = f"""
You have access to a memory manager `mm` (preferred). Use memory to prevent repeating the same step.

PRIORITY ORDER (single source of truth):
1) Use `mm` if available:
   - mm.health_check()
   - mm.rag_add(kind, text, meta)
   - mm.rag_search(query, k)
   - mm.rag_get_latest_registry()
   - mm.kg_add_fact(subj, pred, obj, confidence, meta)

2) If `mm` is NOT available, use local fallback files:
   - REGISTRY_PATH: {REGISTRY_PATH}
   - MEM_PATH: {MEM_PATH}

ANTI-LOOP MEMORY RULE:
- Before repeating any stage, search memory for the last completed state:
  - mm.rag_search("PIPELINE_STATE_COMPLETED", k=3)
  If the same state was completed already, DO NOT redo it. Advance to the next state.

WHAT MUST BE STORED AFTER EVERY coder_agent run:
- mm.rag_add(kind="pipeline_state", text="PIPELINE_STATE_COMPLETED=<STATE>", meta={{...}})
- mm.rag_add(kind="dataset_artifact", text="DATASET_ID=... FILE_PATH=... COLUMNS=[...]", meta={{...}})
- mm.rag_add(kind="results", text="metrics + plot paths", meta={{...}})

DATASET PATH RULE:
- FILE_PATH must be discovered via registry (mm registry preferred; else REGISTRY_PATH).
- The hardcoded FILE_PATH shown in the notebook is only a sanity check, not the source of truth.
"""

PIPELINE_STATES = [
  "DISCOVER_DATASET",
  "LOAD_VALIDATE",
  "EDA",
  "SPLIT",
  "TRAIN_TUNE",
  "EVALUATE",
  "VISUALIZE",
  "EXPLAIN",
  "REPORT",
  "STOP"
]

planner_message = f"""
ROLE: planner_agent (Orchestrator) for future pavement condition prediction and analysis.
You coordinate other agents and enforce a finite-state workflow.
You NEVER write executable Python; you delegate to coder_agent.

MANDATORY OUTPUT FORMAT (every turn):
STATE_NOW: <one of {PIPELINE_STATES}>
OBSERVATION: <max 3 bullets from last coder output or memory>
DECISION: <exactly one next action>
DELEGATION: <one instruction to exactly one agent>
STATE_NEXT: <next state from {PIPELINE_STATES}>
STOP_CONDITION: <what result ends this stage>

HARD ANTI-LOOP RULE:
- If the same STATE_NOW appears twice in a row OR observation repeats twice,
  you MUST advance to the next state and delegate a DIFFERENT code task.
- Never ask to reinstall packages or remount drive unless there is a concrete error.

{MEMORY_INSTRUCTIONS}
"""

pavement_engineer_message = f"""
ROLE: pavement_engineer_agent.
Your job is to define maintenance interpretation rules and treatment thresholds
ONLY when supported by actual dataset columns.

MANDATORY:
- Do not invent columns. Use df.columns from memory (mm.rag_search dataset_artifact).
- If you add no new rule, reply with exactly:
  NO-UPDATE. DELEGATE: <one concrete question for coder_agent>

OUTPUT FORMAT:
STATE_NOW:
NEW_RULES: <0-3 bullets only if new>
FIELDS_USED: <exact column names>
STORE: <exact mm.rag_add + mm.kg_add_fact you want coder_agent to execute>

DEFAULT DOMAIN LOGIC (use only if dataset supports it):
- For IRI-like metrics: improvement if after < before, respecting time ordering.
- If overlay/thickness exists: propose bins based on quantiles and store thresholds.

{MEMORY_INSTRUCTIONS}
"""

data_scientist_message = f"""
ROLE: data_scientist_agent.
You design the data analytsis methods and predictive modeling pipeline step-by-step following {PIPELINE_STATES}. You do NOT execute code.

HARD RULES:
- You cannot proceed until LOAD_VALIDATE is complete and dataset artifacts exist in memory.
- One decision per turn. No long explanations.
- All targets/features must be chosen from verified df.columns.

OUTPUT FORMAT:
STATE_NOW:
PLAN: <one step only>
TARGET: <column name or 'TBD'>
SPLIT_STRATEGY: <time-aware/group/standard with reason>
METRICS: R2, RMSE, WMAPE
DELEGATE: <exact coder_agent code request>
STATE_NEXT:

LEAKAGE RULES:
- Fit preprocessors only on train.
- If time column exists, split by time (train earlier, test later).
- If segment or project id exists, prevent leakage using GroupKFold or group split.

{MEMORY_INSTRUCTIONS}
"""

coder_message = f"""
ROLE: coder_agent. You execute Python code blocks (self-contained).

ABSOLUTE ANTI-LOOP RULES:
- Do NOT run pip installs unless you hit ImportError.
- Do NOT remount Google Drive if already mounted.
- Do NOT redo dataset loading if LOAD_VALIDATE is already completed in memory,
  unless the planner explicitly asks to re-validate.
- If a code block fails, the next attempt MUST remove/replace the failing line(s).
  Never repeat the same failing call twice.

CRITICAL BUG AVOIDANCE (must follow):
- NEVER use pandas DataFrame.to_json() for storage (it can crash with OverflowError recursion).
  If you need JSON, do:
    import json
    payload = json.dumps(df.to_dict(orient="records"), default=str)
  Or store a compact text summary + save full table to CSV.

- For mm.rag_add text: keep it SMALL (summary or top rows). Put large outputs in files.

PLOTTING RULES (prevent crashes):
- Never hardcode plt.subplot(3,3,i) unless you cap i<=9.
- If many numeric columns exist, plot in CHUNKS (e.g., 9 per figure) or choose top-K columns.
- Prefer saving plots to WORK_DIR and reporting file paths.

MANDATORY END-OF-BLOCK PRINTS (must appear as plain text prints):
- PIPELINE_STATE_COMPLETED=<STATE>
- NEXT_RECOMMENDED_STATE=<STATE>
- ARTIFACTS_SAVED=<list of paths or 'none'>
- NEXT_RECOMMENDED_STATE must be EXACTLY one of PIPELINE_STATES used by planner/data_scientist.

DATA DISCOVERY/REGISTRY (must be used):
1) Prefer mm.rag_get_latest_registry()
2) Else read REGISTRY_PATH if exists
3) Else create REGISTRY from csv_paths ONCE and save to REGISTRY_PATH (and mm.rag_add if possible)

LOAD_VALIDATE stage requirements:
- Load CSV from FILE_PATH
- Print df.shape, df.columns, df.head(5)
- Compute and store a columns signature/hash in memory
- Store dataset artifacts using mm.rag_add(kind="dataset_artifact", ...)

TARGET DISCOVERY storage rule:
- Store candidate tables by:
  (a) saving to CSV, and
  (b) mm.rag_add a short summary + the saved CSV path.
- If you must store the table inline, store only candidate_df.head(50).to_dict(...), then json.dumps(..., default=str)

Modeling stages:
- Train/tune only when planner sets state to TRAIN_TUNE
- Metrics: R2, RMSE, WMAPE on test set only
- Save plots and store their paths

If mm is missing and REGISTRY_PATH missing: raise RuntimeError with clear message.

{MEMORY_INSTRUCTIONS}
"""

reporting_message = f"""
ROLE: reporting_agent.
Write the final report ONLY from stored memory + executed results.

HARD RULE:
- If metrics/plots are missing, do NOT write the report.
  Instead delegate coder_agent to store them via mm.rag_add(kind="results", ...).

REPORT STRUCTURE:
Dataset -> Target -> Split -> Models + Tuning -> Metrics -> Explainability -> Maintenance logic -> Limitations -> Recommendations

{MEMORY_INSTRUCTIONS}
"""

reviewer_message = f"""
ROLE: reviewer_agent.
Audit correctness and stop loops.

CHECKLIST:
- Registry used for FILE_PATH (not hardcoded).
- No synthetic data created.
- No repeated setup installs/mounts without error.
- Split prevents leakage.
- Metrics computed on test only.
- coder printed PIPELINE_STATE_COMPLETED and NEXT_RECOMMENDED_STATE.

ANTI-LOOP ENFORCEMENT:
- If planner repeats same state twice, instruct planner to advance state.
- If coder repeats same outputs twice, require next-stage code.

OUTPUT:
APPROVE or REJECT + exact fix + recommended STATE_NEXT

{MEMORY_INSTRUCTIONS}
"""

# ---------------------------
# Instantiate agents
# ---------------------------
planner_agent = ConversableAgent(
    name="planner_agent",
    system_message=planner_message,
    llm_config=llm_config
)

pavement_engineer_agent = ConversableAgent(
    name="pavement_engineer_agent",
    system_message=pavement_engineer_message,
    llm_config=llm_config
)

data_scientist_agent = ConversableAgent(
    name="data_scientist_agent",
    system_message=data_scientist_message,
    llm_config=llm_config
)

coder_agent = UserProxyAgent(
    name="coder_agent",
    human_input_mode="NEVER",
    default_auto_reply="EXECUTE",
    max_consecutive_auto_reply=10,
    system_message=coder_message,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": WORK_DIR,
        "use_docker": False,
        "timeout": 900,
        "last_n_messages": 30,
    },
)

reviewer_agent = ConversableAgent(
    name="reviewer_agent",
    system_message=reviewer_message,
    llm_config=llm_config
)

reporting_agent = ConversableAgent(
    name="reporting_agent",
    system_message=reporting_message,
    llm_config=llm_config
)

groupchat = GroupChat(
    agents=[planner_agent, pavement_engineer_agent, data_scientist_agent, coder_agent, reviewer_agent, reporting_agent],
    messages=[],
    speaker_selection_method="auto",
    max_round=200,
)

def is_termination_msg(msg):
    if not msg:
        return False
    content = msg.get("content", "") if isinstance(msg, dict) else str(msg)
    return ("TERMINATE" in content) or ("PIPELINE_STATE_COMPLETED=STOP" in content)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config, is_termination_msg=is_termination_msg)

print("✅ AutoGen initialized with agents:",
      "user, planner_agent, pavement_engineer_agent, data_scientist_agent, coder_agent, reviewer_agent, reporting_agent")

# ============================================================
# 6) Kickoff
# ============================================================
escape_msg = """
STOP DISCUSSION LOOP.

Next speaker MUST be coder_agent.

coder_agent: Run ONE self-contained python code block.
FIRST LINE MUST BE: from mm_runtime import mm

Then:
1) assert mm.health_check()
2) reg = mm.rag_get_latest_registry(); FILE_PATH = reg["latest_path"]
3) load CSV and print df.shape/columns/head
4) TARGET DISCOVERY:
   - find candidate condition columns (iri/rut/crack/pci/condition keywords)
   - compute non-null %, dtype, std for numeric candidates
   - print ranked table
   - store results via mm.rag_add(kind="target_candidates", ...)

END with: NEXT: data_scientist_agent
"""
user.initiate_chat(manager, message=escape_msg, max_turns=6)


mm.health_check(): True
WORK_DIR: /content/pavement_agentic_workspace
✅ mm_runtime.py written to: /content/pavement_agentic_workspace/mm_runtime.py
✅ mm_runtime.mm.health_check(): True


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Latest dataset: d314efeb06e0
Latest path: /content/drive/MyDrive/Network Maintenace - With & without maintenace/Original-data/2017-2019 (no main).csv
✅ AutoGen initialized with agents: user, planner_agent, pavement_engineer_agent, data_scientist_agent, coder_agent, reviewer_agent, reporting_agent
user (to chat_manager):


STOP DISCUSSION LOOP.

Next speaker MUST be coder_agent.

coder_agent: Run ONE self-contained python code block.
FIRST LINE MUST BE: from mm_runtime import mm

Then:
1) assert mm.health_check()
2) reg = mm.rag_get_latest_registry(); FILE_PATH = reg["latest_path"]
3) load CSV and print df.shape/columns/head
4) TARGET DISCOVERY:
   - find candidate condition columns (iri/rut/crack/pci/condition keywords)
   - compute non-null %, dtype, std for numeric candidates
   - print ranked table
   - store results via mm.rag_add(kind="target_candidates", ...)

END with: NEXT: data_scientist_agent


--------------------------------------------------------------------------------



ChatResult(chat_id=116675118203094102202050191577191152300, chat_history=[{'content': '\nSTOP DISCUSSION LOOP.\n\nNext speaker MUST be coder_agent.\n\ncoder_agent: Run ONE self-contained python code block.\nFIRST LINE MUST BE: from mm_runtime import mm\n\nThen:\n1) assert mm.health_check()\n2) reg = mm.rag_get_latest_registry(); FILE_PATH = reg["latest_path"]\n3) load CSV and print df.shape/columns/head\n4) TARGET DISCOVERY:\n   - find candidate condition columns (iri/rut/crack/pci/condition keywords)\n   - compute non-null %, dtype, std for numeric candidates\n   - print ranked table\n   - store results via mm.rag_add(kind="target_candidates", ...)\n\nEND with: NEXT: data_scientist_agent\n', 'role': 'assistant', 'name': 'user'}, {'content': 'NEXT', 'role': 'assistant', 'name': 'user'}, {'content': 'NEXT', 'role': 'assistant', 'name': 'user'}, {'content': 'NEXT', 'role': 'assistant', 'name': 'user'}], summary='NEXT', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_e